<a href="https://colab.research.google.com/github/dineshgulati933/portneuf_IWRRI/blob/main/ee_python/Pneuf_OpenET_3p_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  Python Script: Pneuf_OpenET_3p_mask.ipynb
#  Author        : Dinesh Gulati
#  Description   : OpenET Data with cropland mask.
# ============================================================

In [ ]:
#5/5/25: This script to fetch all OpenET models including ensemble for multiple years. It will apply reducer to each HUC12 feature
#to calculate mean, 25th, median, 75th in the region.
#The resulted csv for each year will be saved in drive which needs to be processed further to find WY totals or other operations.
#7/22/25: Modified for cultivated and non-cultivated scenarios based on NLCD 2005 LC mask and Portneuf with three sub watersheds

In [ ]:
import ee
import geemap

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-dineshgulati333')

In [ ]:
# Define model collections (only 'et' band selected)
models = {
    'ensb': ee.ImageCollection("OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0").select('et_ensemble_mad'),
    'sebal': ee.ImageCollection("OpenET/GEESEBAL/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'sims': ee.ImageCollection("OpenET/SIMS/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'ptjpl': ee.ImageCollection("OpenET/PTJPL/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'alexi': ee.ImageCollection("OpenET/DISALEXI/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'sseb': ee.ImageCollection("OpenET/SSEBOP/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'metric': ee.ImageCollection("OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0").select('et')
}

In [ ]:
# Band rename dictionary
bandNamesDict = {
    'ensb': 'et_ensb',
    'sebal': 'et_sebal',
    'sims': 'et_sims',
    'ptjpl': 'et_ptjpl',
    'alexi': 'et_alexi',
    'sseb': 'et_sseb',
    'metric': 'et_metric'
}

In [ ]:
#NCLD mask
nlcd = ee.ImageCollection('projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER') \
                            .filter(ee.Filter.date('2005-01-01', '2005-01-31')).first()

crop_mask = nlcd.eq(82).Or(nlcd.eq(81))
non_crop_mask = nlcd.neq(82).Or(nlcd.eq(81))

In [ ]:
aoi = ee.FeatureCollection("projects/ee-dineshgulati333/assets/portneuf3_clean")
aoi_bounds = aoi.geometry().bounds()

# Define the range of years and months
years = list(range(1999, 2025))  # 1999 to 2024
months = list(range(1, 13))  # January to December

# Get projection from an ET image to preserve scale
sample_projection = models['ensb'].first().projection()

In [ ]:
# Map = geemap.Map()
# Map.add_basemap("OpenStreetMap")
# Map.addLayer(aoi, {'color':'red'})
# Map.centerObject(aoi, 8)
# Map.addLayer(crop_mask,)
# Map.addLayer(aoi_bounds,)
# Map

In [ ]:
# List to store tasks
tasks = []

# Outer loop over models
for model_name, model_et in models.items():

    for year in years:
        months_for_year = range(10, 13) if year == 1999 else months
        def create_monthly_composite(month):
            etm = model_et.filter(ee.Filter.calendarRange(year, year, 'year')) \
                          .filter(ee.Filter.calendarRange(month, month, 'month')) \
                          .filterBounds(aoi_bounds) \
                          .mosaic() \
                          .setDefaultProjection(sample_projection) \
                          .set('system:time_start', ee.Date.fromYMD(year, month, 1).millis()) \
                          .set('Year', year, 'Month', month, 'Date', ee.Date.fromYMD(year, month, 1).format('YYYY-MM-dd'))
            etm_crop = etm.updateMask(crop_mask)
            etm_non_crop = etm.updateMask(non_crop_mask)
            etm = etm_crop.rename('et_crop').addBands(etm_non_crop.rename('et_non_crop'))
            return etm

        allMonthlyComp = ee.ImageCollection([create_monthly_composite(m) for m in months_for_year])

        # Stats function → mean + percentiles only
        def Stats(image):
            reducer = ee.Reducer.mean() \
                .combine(ee.Reducer.percentile([25, 50, 75], ['25th', '50th', '75th']), sharedInputs=True)

            stats = image.reduceRegions(
                collection=aoi,
                reducer=reducer,
                scale=sample_projection.nominalScale() # or image.projection().nominalScale()
            )

            stats = stats.map(lambda f: f.set('Date', image.get('Date')))
            return stats

        et_stats = allMonthlyComp.map(Stats).flatten()

        # Selectors for export
        selectors = ['Date', 'UID', 'et_crop_25th', 'et_crop_50th', 'et_crop_75th', 'et_crop_mean',
                     'et_non_crop_25th', 'et_non_crop_50th', 'et_non_crop_75th', 'et_non_crop_mean']
        # Define export task
        task = ee.batch.Export.table.toDrive(
            collection=et_stats,
            description=f'OpenET_{model_name}_monthly_stat_{year}',
            folder='Pneuf_OpenET_3p_mask',
            fileFormat='CSV',
            selectors=selectors
        )

        # Start export
        task.start()
        tasks.append(task)
        print(f"Export started for {model_name}, {year}")

print("\nAll export tasks have been started in parallel!")


Export started for ensb, 1999
Export started for ensb, 2000
Export started for ensb, 2001
Export started for ensb, 2002
Export started for ensb, 2003
Export started for ensb, 2004
Export started for ensb, 2005
Export started for ensb, 2006
Export started for ensb, 2007
Export started for ensb, 2008
Export started for ensb, 2009
Export started for ensb, 2010
Export started for ensb, 2011
Export started for ensb, 2012
Export started for ensb, 2013
Export started for ensb, 2014
Export started for ensb, 2015
Export started for ensb, 2016
Export started for ensb, 2017
Export started for ensb, 2018
Export started for ensb, 2019
Export started for ensb, 2020
Export started for ensb, 2021
Export started for ensb, 2022
Export started for ensb, 2023
Export started for ensb, 2024
Export started for sebal, 1999
Export started for sebal, 2000
Export started for sebal, 2001
Export started for sebal, 2002
Export started for sebal, 2003
Export started for sebal, 2004
Export started for sebal, 2005
Exp